<a href="https://colab.research.google.com/github/dhanyahari07/medium_post/blob/main/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.express as px
from wordcloud import WordCloud
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_json("/content/drive/MyDrive/sarcasm/Sarcasm_Headlines_Dataset_v2.json", lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


In [ ]:
# Let's use regex on the "article_link" column to get the source data.
df["source"] = df["article_link"].apply(lambda string: re.findall('[a-zA-Z]+', string)[2] )
df = df.drop(['article_link'],axis = 1)

In [ ]:
import re
df['headline'] = df['headline'].str.lower()
df['headline'] = df['headline'].apply(lambda x: re.sub('[^a-zA-Z\s]+|X{2,}', '', x))

# [@\^&\*$] matches on the characters, you want to replace
#\S+matches on hash tags
# \S+[a-z0-9]\.(com|net|org) matches on domain names

df['headline'] = df['headline'].apply(lambda x: re.sub("[@\^&\*$]|#\S+|\S+[a-z0-9]\.(com|net|org)"," ",x))

In [ ]:
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not",
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Expanding Contractions in the title, text

In [ ]:
# Expanding Contractions in the title, text
df['headline'] = df['headline'].apply(lambda x:expand_contractions(x))

In [ ]:
df['CharCount'] = df['headline'].apply(lambda x: len(x))
df['WordCount'] = df['headline'].apply(lambda x: len(x.split()))

In [ ]:

df.head(10)

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep totally nails why congress is falling ...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word stream...,https://www.theonion.com/mother-comes-pretty-c...
5,0,my white inheritance,https://www.huffingtonpost.com/entry/my-white-...
6,0,ways to file your taxes with less stress,https://www.huffingtonpost.com/entry/5-ways-to...
7,1,richard bransons globalwarming donation nearly...,https://www.theonion.com/richard-bransons-glob...
8,1,shadow government getting too large to meet in...,https://politics.theonion.com/shadow-governmen...
9,0,lots of parents know this scenario,https://www.huffingtonpost.comhttp://pubx.co/6...


In [ ]:
df["source"].value_counts()

huffingtonpost    14985
theonion          13634
Name: source, dtype: int64

In [ ]:
df["is_sarcastic"].value_counts()

0    14985
1    13634
Name: is_sarcastic, dtype: int64

In [ ]:
def tokenize(text):
    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [ ]:
X = df.headline
Y = df.is_sarcastic
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:

import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Instantiate transformers and classifier
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf = RandomForestClassifier()


# Fit and/or transform each to the training data
# Hint: you can use the fit_transform method
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)

# Fit or train the classifier
clf.fit(X_train_tfidf, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


RandomForestClassifier()

In [ ]:
# Transform test data
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)

# Predict test labels
y_pred = clf.predict(X_test_tfidf)

In [ ]:
print(classification_report(y_test.values, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.82      0.82      4916
           1       0.80      0.79      0.80      4529

    accuracy                           0.81      9445
   macro avg       0.81      0.81      0.81      9445
weighted avg       0.81      0.81      0.81      9445



In [ ]:

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8067815071519029

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
# prepocessing the headlines

#import numpy as np
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# the dataset is saved as json file and we will use json module to load it into our workspace
import json
data_path = '/content/drive/MyDrive/sarcasm/Sarcasm_Headlines_Dataset_v2.json'

def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

datastore = list(parse_data(data_path))

In [ ]:
# initialize lists
sentences = []
labels = []
urls = []

# append elements into each list
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [ ]:
#Hyperparameters
training_size = 20000
vocab_size = 10000
max_length = 32
embedding_dim = 16
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.33, random_state=42)

In [ ]:



#split the sentences
X_train = sentences[0:training_size]
X_test = sentences[training_size:]

#split the labels
y_train = labels[0:training_size]
y_test = labels[training_size:]

In [ ]:


#initialize the tokinizer class
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

#generate the word index dictionary
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

#generate and pad the training sequence
training_sequences = tokenizer.texts_to_sequences(X_train)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

#generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

#convert the labels lists into numpy arrays
training_labels = np.array(y_train)
testing_labels = np.array(y_test)

In [ ]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length))
model.add(LSTM(units=100))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
#model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)



Epoch 1/30
600/600 - 41s - loss: 0.4296 - accuracy: 0.7896 - val_loss: 0.3560 - val_accuracy: 0.8445 - 41s/epoch - 68ms/step
Epoch 2/30
600/600 - 38s - loss: 0.2423 - accuracy: 0.9067 - val_loss: 0.4298 - val_accuracy: 0.8395 - 38s/epoch - 63ms/step
Epoch 3/30
600/600 - 39s - loss: 0.1716 - accuracy: 0.9359 - val_loss: 0.3990 - val_accuracy: 0.8449 - 39s/epoch - 65ms/step
Epoch 4/30
600/600 - 38s - loss: 0.1254 - accuracy: 0.9535 - val_loss: 0.4805 - val_accuracy: 0.8401 - 38s/epoch - 63ms/step
Epoch 5/30
600/600 - 39s - loss: 0.0969 - accuracy: 0.9629 - val_loss: 0.5085 - val_accuracy: 0.8321 - 39s/epoch - 65ms/step
Epoch 6/30
600/600 - 38s - loss: 0.0785 - accuracy: 0.9712 - val_loss: 0.6633 - val_accuracy: 0.8336 - 38s/epoch - 63ms/step
Epoch 7/30
600/600 - 38s - loss: 0.0611 - accuracy: 0.9784 - val_loss: 0.7302 - val_accuracy: 0.8344 - 38s/epoch - 63ms/step
Epoch 8/30
600/600 - 36s - loss: 0.0469 - accuracy: 0.9834 - val_loss: 0.7995 - val_accuracy: 0.8341 - 36s/epoch - 59ms/step


In [ ]:
history

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(testing_padded, testing_labels)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

270/270 [==============================] - 4s 13ms/step - loss: 1.3542 - accuracy: 0.8226
Test Loss: 1.3542133569717407, Test Accuracy: 0.8226012587547302


In [ ]:
# text preprocessing

In [ ]:
# FINAL

# FINAL

In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
df = pd.read_json("/content/drive/MyDrive/sarcasm/Sarcasm_Headlines_Dataset_v2.json", lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
df['headline']

0        thirtysomething scientists unveil doomsday clo...
1        dem rep totally nails why congress is falling ...
2          eat your veggies  deliciously different recipes
3        inclement weather prevents liar from getting t...
4        mother comes pretty close to using word stream...
                               ...                        
28614         jews to celebrate rosh hashasha or something
28615    internal affairs investigator disappointed con...
28616    the most beautiful acceptance speech this week...
28617    mars probe destroyed by orbiting spielberggate...
28618                   dad clarifies this not a food stop
Name: headline, Length: 28619, dtype: object

In [ ]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


In [ ]:
df['headline'] = df['headline'].str.lower()
df['headline'] = df['headline'].apply(lambda x: re.sub('[^a-zA-Z\s]+', '', x))

# [@\^&\*$] matches on the characters, you want to replace
#\S+matches on hash tags
# \S+[a-z0-9]\.(com|net|org) matches on domain names

df['headline'] = df['headline'].apply(lambda x: re.sub("[@\^&\*$]|#\S+|\S+[a-z0-9]\.(com|net|org)"," ",x))

In [ ]:
cleaned_text = lambda x: re.sub('[^a-zA-Z\s]+|X{2,}', '', x)
result = cleaned_text("This is some text with 123 XXXs!!!")
print(result)

This is some text with  s


In [ ]:
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not",
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Expanding Contractions in the title, text

In [ ]:
# Expanding Contractions in the title, text
df['headline'] = df['headline'].apply(lambda x:expand_contractions(x))

In [ ]:
df["is_sarcastic"].value_counts()

0    14985
1    13634
Name: is_sarcastic, dtype: int64

In [ ]:
#Hyperparameters
training_size = 20000
vocab_size = 10000
max_length = 32
embedding_dim = 16
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [ ]:
X = df.headline
Y = df.is_sarcastic
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:


#initialize the tokinizer class
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

#generate the word index dictionary
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

#generate and pad the training sequence
training_sequences = tokenizer.texts_to_sequences(X_train)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

#generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

#convert the labels lists into numpy arrays
training_labels = np.array(y_train)
testing_labels = np.array(y_test)

In [ ]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length))
model.add(LSTM(units=100))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
#model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)



Epoch 1/30
600/600 - 48s - loss: 0.4512 - accuracy: 0.7789 - val_loss: 0.3763 - val_accuracy: 0.8431 - 48s/epoch - 80ms/step
Epoch 2/30
600/600 - 35s - loss: 0.2530 - accuracy: 0.9029 - val_loss: 0.3642 - val_accuracy: 0.8365 - 35s/epoch - 59ms/step
Epoch 3/30
600/600 - 36s - loss: 0.1792 - accuracy: 0.9326 - val_loss: 0.5086 - val_accuracy: 0.8329 - 36s/epoch - 59ms/step
Epoch 4/30
600/600 - 37s - loss: 0.1334 - accuracy: 0.9500 - val_loss: 0.5809 - val_accuracy: 0.8380 - 37s/epoch - 62ms/step
Epoch 5/30
600/600 - 34s - loss: 0.1013 - accuracy: 0.9622 - val_loss: 0.5536 - val_accuracy: 0.8276 - 34s/epoch - 56ms/step
Epoch 6/30
600/600 - 33s - loss: 0.0745 - accuracy: 0.9734 - val_loss: 0.7280 - val_accuracy: 0.8307 - 33s/epoch - 55ms/step
Epoch 7/30
600/600 - 33s - loss: 0.0579 - accuracy: 0.9801 - val_loss: 0.8275 - val_accuracy: 0.8239 - 33s/epoch - 55ms/step
Epoch 8/30
600/600 - 34s - loss: 0.0447 - accuracy: 0.9847 - val_loss: 0.7180 - val_accuracy: 0.8264 - 34s/epoch - 57ms/step


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(testing_padded, testing_labels)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

296/296 [==============================] - 7s 24ms/step - loss: 1.0846 - accuracy: 0.8142
Test Loss: 1.084608554840088, Test Accuracy: 0.8141874074935913
